In [1]:
from scipy.integrate import *
import pandas as pd
import numpy as np
from lmfit import minimize, Parameters, Parameter, report_fit

In [2]:
#California population obtained from google
Population = 39,538,223

In [3]:
df = pd.read_csv('covid19casesdemographics.csv')

In [4]:
age_df = df[df['demographic_category']== 'Age Group']

In [5]:
age_df['demographic_value'].unique()

array(['0-17', '18-49', '50-64', '65+', 'Missing', 'Total', 'missing'],
      dtype=object)

In [6]:
age_df = age_df[age_df.demographic_value.isin(['0-17', '18-49', '50-64', '65+'])]

In [7]:
#California population obtained from google
Population = 39538223
Days = len(age_df['report_date'].unique())

In [8]:
#obtain I data

ages = ['0-17', '18-49', '50-64', '65+']

I = dict()
for i in ages:
    age_group = age_df[age_df['demographic_value'] == i]
    Ii = []
    
    #first day
    Ii.append(age_group['total_cases'].iloc[0])
    
    #for first day 1-7, do average of its neighbors
    for k in range(1, 7):
        Ii.append(np.mean(age_group['total_cases'].iloc[k-1:k+1]))
    
    #moving average of windows of t = 15
    for k in range(7, len(age_group['total_cases'])-7):
        Ii.append(np.mean(age_group['total_cases'].iloc[k-7:k+7]))
    
    #for last -7 to -1 days do average of its neighbors
    for k in range(len(age_group['total_cases'])-7, len(age_group['total_cases'])-1):
        Ii.append(np.mean(age_group['total_cases'].iloc[k-1:k+1]))
    #for last day, just append the value
    Ii.append(age_group['total_cases'].iloc[Days-1])
    I[i] = Ii


In [9]:
#Obtain Recovered Data
Recovery_days = 14

R = dict()
for i in ages:
    age_group = age_df[age_df['demographic_value'] == i]
    Ri = []
    for k in range(0, Recovery_days):
        Ri.append(0)
    
    for k in range(14, Days):
        Ri.append(age_group['total_cases'].iloc[k-14])
    R[i] = Ri
    

In [10]:
#obtain Death data

ages = ['0-17', '18-49', '50-64', '65+']

D = dict()
for i in ages:
    age_group = age_df[age_df['demographic_value'] == i]
    Di = []
    
    #first day
    Di.append(age_group['deaths'].iloc[0])
    
    #for first day 1-7, do average of its neighbors
    for k in range(1, 7):
        Di.append(np.mean(age_group['deaths'].iloc[k-1:k+1]))
    
    #moving average of windows of t = 15
    for k in range(7, len(age_group['deaths'])-7):
        Di.append(np.mean(age_group['deaths'].iloc[k-7:k+7]))
    
    #for last -7 to -1 days do average of its neighbors
    for k in range(len(age_group['deaths'])-7, len(age_group['deaths'])-1):
        Di.append(np.mean(age_group['deaths'].iloc[k-1:k+1]))
    #for last day, just append the value
    Di.append(age_group['deaths'].iloc[Days-1])
    D[i] = Di


In [11]:
#obtain Susceptible data
percentages = age_df['percent_of_ca_population'].unique()
age_pop = dict(zip(ages, percentages))

S = dict()
for age in ages:
    Si = []
    for i, j, k in zip(I[age], R[age], D[age]):
        Si.append(age_pop[age]*Population/100-i-j-k)
    S[age] = Si